### Read in word list

In [1]:
import src.utils.wordarrays as wordarrays
import src.utils.git as gitutil
bpath = gitutil.get_root()

In [2]:
word_list_name = 'twl06.txt'

word_list_file = bpath + '/data/processed/' + word_list_name

with open(word_list_file, 'r') as f:
    word_list = f.read().splitlines()

### Initialise

In [3]:
possible_words = word_list

### Read in pre-calculated full cross-check

In [4]:
import numpy as np

In [5]:
full_cross_check = np.load(bpath + '/data/processed/full_cross_check_bin.npy')

## Set up functions to carry out game steps

In [6]:
def find_min_entropy_word(cross_checks):

    entropies = [wordarrays.calc_entropy(check) for check in cross_checks]

    min_entropy_word_index = entropies.index(max(entropies))

    return(min_entropy_word_index)

In [7]:
def array_from_record(record):

    binary_index = full_cross_check[record[0],:]==record[1]

    return(binary_index)

In [8]:
def make_outcomes_tuple(outcomes):

    outcomes_tuple = tuple(array_from_record(outcome) for outcome in outcomes)

    return(outcomes_tuple)

In [9]:
def determine_possible_from_outcomes(outcomes):

    outcomes_tuple = make_outcomes_tuple(outcomes)

    index_of_possible = np.where(np.all(np.dstack(outcomes_tuple), axis=2))[1]

    return(index_of_possible)

Pre-process first guess to save time in each game

In [10]:
min_entropy_first_guess = find_min_entropy_word(full_cross_check)

In [11]:
def make_guess(target_index, outcomes):

    if len(outcomes) > 0:
        possible_from_outcomes = determine_possible_from_outcomes(outcomes)

        #calculate minimum entropy across all words
        min_entropy_word = find_min_entropy_word(full_cross_check[:,possible_from_outcomes])
        min_entropy = wordarrays.calc_entropy(full_cross_check[min_entropy_word,possible_from_outcomes])
        
        # calculate minimum entropy across possible words
        min_entropy_word_poss = possible_from_outcomes[find_min_entropy_word(full_cross_check[:,possible_from_outcomes][possible_from_outcomes,:])]
        min_poss_entropy = wordarrays.calc_entropy(full_cross_check[min_entropy_word_poss,possible_from_outcomes])

        # choose a possible word if it has minimum entropy overall
        if min_poss_entropy == min_entropy:
            min_entropy_word = min_entropy_word_poss

    else:
        min_entropy_word = min_entropy_first_guess

    outcome = full_cross_check[min_entropy_word,target_index]

    outcome_tuple = tuple((min_entropy_word, outcome))

    outcomes.append(outcome_tuple)

    return(outcomes)

In [12]:
def play_game(target_word, show_guesses=True):

    target_index = possible_words.index(target_word)

    outcomes = make_guess(target_index, [])

    if show_guesses is True:
            print(outcomes)

    while(outcomes[-1][-1]!=992):
        outcomes = make_guess(target_index, outcomes)
        
        if show_guesses is True:
            print(outcomes)
    
    final_guess = determine_possible_from_outcomes(outcomes)

    number_guesses = len(outcomes)

    return possible_words[final_guess[0]], number_guesses

In [13]:
play_game('words')

[(7678, 160)]
[(7678, 160), (2072, 272)]
[(7678, 160), (2072, 272), (6680, 22)]
[(7678, 160), (2072, 272), (6680, 22), (8722, 992)]


('words', 4)